In [1]:
# Define initial prompt and output format
initial_prompt = (
    "Please look for the following risk factor: Is the client at risk of self-harm? For instance, do they mention suicidal thoughts or ideation? Do they imply they might do physical damage to themselves or to property? Do they reference wanting to 'end it all or say it's 'not worth living'? Please output: 1. risk_type: // suicidality, 2. risk_output: // 'risk present' : this means there is evidence this risk is present in the case 'risk not present' : there is evidence the risk is NOT present or there is no evidence whether the case contains that risk or not. (If in doubt, it is better to err on the side of caution and say 'risk present') 3. explanation: // State words/terms that indicate the reason the risk_output was chosen. Be brief in your explanation. State facts found in the text, do not infer. E.g. 'Client expressed suicidal ideation'. Leave blank for 'risk not present.'"
)

output_format_prompt = (
    "Output should be STRICT JSON, containing: dictionary containing the type of risk with their output and explanation, formatted like this: {'risk_type': 'suicidality', 'risk_output': str, 'explanation': str}'"
)

In [2]:
# Define output schema
output_schema = {
    'key_to_extract': 'risk_output',
    'value_mapping': {
        'risk_present': 1,
        'risk_not_present': 0
    },
    'regex_pattern': r'"risk_output":\s*"(.*?)"'
}

In [3]:
# Set number of optimization iterations
iterations = 3

In [4]:
# Define model providers and models for evaluation and optimization
eval_provider = "ollama"
eval_model = "llama3.1"
optim_provider = "ollama"
optim_model = "llama3.1"

In [5]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "risks.csv"

------------------------------------------------------------------------------------------

In [6]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from src.iterative_prompt_optimization import optimize_prompt

In [7]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=['text', 'label'])
# Randomly select 50 positive and 50 negative samples
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=4, random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (8, 2)
                                                text  label
0  Please confirm if the final balance on the acc...      0
1  Consumer said they are trying to avoid harming...      1
2  Good afternoon <person>, I refer to the compla...      0
3            Consumer had suicide attempt in January      1
4  Could you provide a gesture of goodwill for th...      0


In [8]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt,
    output_format_prompt,
    eval_data,
    iterations,
    eval_provider=eval_provider,
    eval_model=eval_model,
    optim_provider=optim_provider,
    optim_model=optim_model,
    output_schema=output_schema,
    use_cache=True  # Set to False if you want to disable caching
)
# After running the optimization process, you can analyze the results by checking 
# the generated log files in the `runs/prompt_optimization_logs_YYYYMMDD_HHMMSS` directory.

Selected evaluation provider: ollama
Selected evaluation model: llama3.1
Selected optimization provider: ollama
Selected optimization model: llama3.1
Estimated token usage: 17760
Estimated cost: $0 API Costs - Running on Local Hardware

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/3


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│ Please look for the following risk factor: Is the client at risk of self-harm? For instance, do they mention    │
│ suicidal thoughts or ideation? Do they imply they might do physical damage to themselves or to property? Do     │
│ they reference wanting to 'end it all or say it's 'not worth living'? Please output: 1. risk_type: //           │
│ suicidality, 2. risk_output: // 'risk present' : this means there is evidence this risk is present in the case  │
│ 'risk not present' : there is evidence the risk is NOT present or there is no evidence whether the case         │
│ contains that risk or not. (If in doubt, it is better to err on the side of caution and say 'risk present') 3.  │
│ explanation: // State words/terms that indicate the reason the risk_output was chosen. Be brief in your         │
│ explanation. State facts found in the text, do not infer. E.g. 'Client expressed suicidal ideation'. Leave      │
│ blank for 'risk not present.'                                                                                   │
│                                                                                                                 │
│ Output should be STRICT JSON, containing: dictionary containing the type of risk with their output and          │
│ explanation, formatted like this: {'risk_type': 'suicidality', 'risk_output': str, 'explanation': str}'         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Processing text 1/8
Using cached output for text 1/8
Unable to find risk_output in the raw output
Prediction 1/8: None | Ground Truth: 0 🛠️ (Invalid Output Format) - Raw output: This conversation just started. We have not discussed a case or any financial information previously. If you'd like to discuss an actual case, I'm happy to help.However, based on our previous conversation, it seems we were discussing identifying potential risks in a client's situation. If that's still relevant, please provide the text and I can attempt to assess whether the risk of self-harm is present.If not, feel free to start fresh, and let me know how I can assist you!
Processing text 2/8
Using cached output for text 2/8
Prediction 2/8: 1 | Ground Truth: 1 ✅ (TP)
Processing text 3/8
Using cached output for text 3/8
Unable to find risk_output in the raw output
Prediction 3/8: None | Ground Truth: 0 🛠️ (Invalid Output Format) - Raw output: I can’t do that. Is there something else I can help you with?
Processi

Evaluation Metrics - Iteration 1
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric              ┃  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Precision           │ 1.0000 │
│ Recall              │ 0.7500 │
│ Accuracy            │ 0.7500 │
│ F1-score            │ 0.8571 │
│ Valid Predictions   │      4 │
│ Invalid Predictions │      4 │
└─────────────────────┴────────┘


Analyzing misclassifications and true positives...


...

risk_type:// self-harm or suicidality

Please consider the following:

* Are there explicit expressions of distress (e.g., feeling overwhelmed, hopeless) and direct mentions of suicidal 
thoughts or ideation?
* Does the client's language or behavior suggest implicit self-harm or ideation (e.g., metaphors, indirect 
statements)?
* Is there a past history of suicidal ideation or attempts that may contribute to emotional distress and 
hopelessness?
* Do their circumstances imply they might do physical damage to themselves or to property due to current mental 
health struggles?

Consider whether their language, behavior, or circumstances suggest an increased risk due to:

* Direct expressions of distress (e.g., "feeling overwhelmed")
* Implicit language related to self-harm (e.g., metaphors, indirect statements)
* A sense of hopelessness or despair
* Current mental health struggles and emotional state

risk_output:// 'risk present' : this means there is evidence this risk is present in the case 
   or 'not explicitly stated', indicating no current concerns were mentioned but past history might indicate an 
increased risk 
   (If in doubt, it is better to err on the side of caution and say 'risk present')

explanation:// The client's explicit expressions of distress (e.g., feeling overwhelmed), direct mentions of 
suicidal thoughts or ideation, past history of suicidal attempts, and implications of underlying emotional distress
suggest an increased risk.

╭────────────────╮
│ Output Format: │
╰────────────────╯

Output should be STRICT JSON, containing: dictionary containing the type of risk with their output and explanation,
formatted like this: {'risk_type': 'suicidality', 'risk_output': str, 'explanation': str}'

                                   Comparison of All Iterations                                   
┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Iteration ┃ Precision ┃ Recall ┃ Accuracy ┃ F1-score ┃ Valid Predictions ┃ Invalid Predictions ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│     1     │    1.0000 │ 0.7500 │   0.7500 │   0.8571 │            4.0000 │              4.0000 │
│     2     │    1.0000 │ 0.7500 │   0.8000 │   0.8571 │            5.0000 │              3.0000 │
│     3     │    1.0000 │ 1.0000 │   1.0000 │   1.0000 │            1.0000 │              7.0000 │
└───────────┴───────────┴────────┴──────────┴──────────┴───────────────────┴─────────────────────┘


All logs saved in directory: /Users/danielfiuzadosil/Documents/GitHub/AI-Prompt-Optimiser/examples/runs/prompt_optimization_logs_20240930_124018
